# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4765, uptime 0:00:22
mini_beamline                    RUNNING   pid 4766, uptime 0:00:22
random_walk                      RUNNING   pid 4767, uptime 0:00:22
random_walk_horiz                RUNNING   pid 4768, uptime 0:00:22
random_walk_vert                 RUNNING   pid 4769, uptime 0:00:22
simple                           RUNNING   pid 4770, uptime 0:00:22
thermo_sim                       RUNNING   pid 4771, uptime 0:00:22
trigger_with_pc                  RUNNING   pid 4772, uptime 0:00:22


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 1     Time: 2020-05-18 14:32:49
Persistent Unique Scan ID: 'ec1f18a9-e755-4f49-9ba1-f5afa0a6ebdf'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 14:32:49.5 |     -1.000 |      93625 |
|         2 | 14:32:49.5 |     -0.500 |      94293 |
|         3 | 14:32:49.5 |      0.000 |      95400 |
|         4 | 14:32:49.5 |      0.500 |      95322 |


|         5 | 14:32:49.6 |      1.000 |      93293 |


+-----------+------------+------------+------------+
generator scan ['ec1f18a9'] (scan num: 1)





('ec1f18a9-e755-4f49-9ba1-f5afa0a6ebdf',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/ec1f18a9-e755-4f49-9ba1-f5afa0a6ebdf-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 2     Time: 2020-05-18 14:32:49
Persistent Unique Scan ID: '9b52d6f3-012d-4310-aab1-d3bf3dfe697a'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 14:32:50.1 |          -1 | 1055741038 |


|         2 | 14:32:50.2 |          -0 | 1063694693 |


|         3 | 14:32:50.3 |           0 | 1069082703 |


|         4 | 14:32:50.4 |           0 | 1074993868 |


|         5 | 14:32:50.5 |           1 | 1080662407 |
+-----------+------------+-------------+------------+
generator scan ['9b52d6f3'] (scan num: 2)





('9b52d6f3-012d-4310-aab1-d3bf3dfe697a',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-18 14:32:50
Persistent Unique Scan ID: '76ae0c79-1b89-4be5-ada0-b9f0fb3a7d79'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 14:32:51.0 | 1105567581 |


|         2 | 14:32:52.0 | 1128596924 |


|         3 | 14:32:53.0 | 1076784448 |


...

|        28 | 14:33:18.0 | 1058163264 |


|        29 | 14:33:19.0 | 1111736657 |


|        30 | 14:33:20.0 | 1126652596 |


+-----------+------------+------------+
generator count ['76ae0c79'] (scan num: 3)





('76ae0c79-1b89-4be5-ada0-b9f0fb3a7d79',)